In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D
from sklearn.model_selection import train_test_split

In [ ]:
#load cleaned data
loaded = np.load('mtg_data_balanced.npz', allow_pickle=True)
X = loaded['X']
y = loaded['Y']

In [ ]:
#confirm correct loading and dimensions
print(f'Feature matrix shape: {X.shape}')
print(f'Label array shape: {y.shape}')

Feature matrix shape: (14500, 96, 1400, 1)
Label array shape: (14500, 11)


In [7]:
y= y.astype(np.float32)

In [9]:
#splotting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Training feature matrix shape: {X_train.shape}')
print(f'Testing feature matrix shape: {X_test.shape}')
print(f'Training label array shape: {y_train.shape}')
print(f'Testing label array shape: {y_test.shape}')

Training feature matrix shape: (11600, 96, 1400, 1)
Testing feature matrix shape: (2900, 96, 1400, 1)
Training label array shape: (11600, 11)
Testing label array shape: (2900, 11)


In [ ]:
#initial model
model = Sequential([
    Conv2D(filters=20, kernel_size=(5,5), strides=(2, 2), input_shape=[96,1400,1], padding='valid', activation='relu'),
    Conv2D(filters=40, kernel_size=(5,5), strides=(2, 2), input_shape=[96,1400,1], padding='valid', activation='relu'),
    Flatten(),
    Dense(10, activation='softmax'),
])
#model.compile(optimizer='adam', loss=', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train[:, 1:], epochs=4, batch_size=32, validation_data=(X_test, y_test[:, 1:]))

c:\Users\olive\OneDrive\Desktop\DS\ANN_DIS\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/4
363/363 ━━━━━━━━━━━━━━━━━━━━ 120s 322ms/step - accuracy: 0.2086 - loss: 4.6209 - val_accuracy: 0.2462 - val_loss: 2.1444
Epoch 2/4
363/363 ━━━━━━━━━━━━━━━━━━━━ 142s 392ms/step - accuracy: 0.4277 - loss: 1.7203 - val_accuracy: 0.2479 - val_loss: 2.2621
Epoch 3/4
363/363 ━━━━━━━━━━━━━━━━━━━━ 183s 339ms/step - accuracy: 0.6974 - loss: 0.9441 - val_accuracy: 0.2117 - val_loss: 2.9217
Epoch 4/4
363/363 ━━━━━━━━━━━━━━━━━━━━ 109s 301ms/step - accuracy: 0.8834 - loss: 0.3797 - val_accuracy: 0.2248 - val_loss: 4.0228


Lessons learned from initial model training and concerns going forward:

The model is severely over fit, and I will need to look into regularization methods. I didn't expect this, because I anticipated the chorus of songs to show up in both sets, resulting in high validation accuracy. That's doesn't seem to be the case. I'm want to try dropout layers and l2 norm regularization. Additionally, this architecture was an arbitrary choice so I want to be more intentional about architecture going forward.

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 698, 20)    │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 347, 40)    │        20,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 291480)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │     2,914,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,806,112 (33.59 MB)

 Trainable params: 2,935,370 (11.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,870,742 (22.40 MB)